# Colab 시작
- colab은 local 환경이 아닌, cloud 환경에서 작동됨. 따라서 **코드를 실행하기 위한 세팅이 따로 필요없음.**
- 앞에 초록색 check 표시가 있는 셀은 이미 실행된 셀임
  - 또 실행해도 상관은 없음
  - `pip install`은 시간이 걸리니, 굳이 또 설치하지 말자.
  - openai api key는 매번 입력하기 번거로움 + 현재 key에 deposit 충전되어있으므로 돈 충전 또 하기 싫으면 그냥 쓰자.
  - 공유하고 있는 누군가가 이미 실행했다는 뜻
- 셀 실행 : `shift` + `enter`


In [ ]:
%%capture
# Libraries are installed to the instance which can only last for max 12 hours. After that, the instance is destroyed.
!pip install langchain
!pip install langchain_openai

In [2]:
import os
import getpass
import ast
import re
import numpy as np
import langchain
from langchain_openai import OpenAIEmbeddings
from urllib.parse import quote

In [3]:
# openai api key
# os.environ["OPENAI_API_KEY"] = getpass.getpass()
os.environ["OPENAI_API_KEY"] = ''

In [4]:
# Embedding 모델 가져오기
embed_model = OpenAIEmbeddings(model='text-embedding-3-large') # dimensions(int): specify vector size

# 입력사항
- 파일을 사용하기 위해서는 코드 실행할 때와 마찬가지로 cloud 환경에 업로드해야함.
- (예정) 파일은 이미 업로드 되어있으므로, 뒤의 `KDS 41 17 00 건축물 내진설계기준.txt` 부분만 수정하고 실행하면 됨

### Issue!
 - query에 오타가 있는 경우는 해결이 되는 것 같으나, query의 질문 형식이 이상하면 결과가 잘 안나오는 경우가 있는듯?
 - query를 조금 더 수정해서 embedding model에 입력할 수 있는 step이 추가되는게 좋을듯

In [40]:
# 사용자 검색어
# query = '유효지방가속도가 머임'
# query = '등가정적해석법에서 밑면전단력 어떻게 구함?'
# query = '층간변위비 어케구함'
# query = 'how to calculate 층간변위비'
# query = '츠ㅇ간변위비'
# query = '층간변위비 어케계산함'
# query = 'base shear'
query = '지압접합 강구조'

# 표시할 결과 갯수
show_result = 5

# 실행
- 위에서 입력한 *검색어*와 *파일 경로*를 바탕으로 **Embedding 생성** 및 **유사도 계산**
- *검색어*와 *파일 경로*가 바뀔 시 재실행해야함!

In [34]:
# pdf 파일 경로
# pdf_path = '/content/converted_txt'
pdf_path = '../../converted_txt'

# 저장된 txt 파일을 읽어서 list로 변환
count = 0
section_list = []
for file in os.listdir(pdf_path):
  if file.endswith('.txt'):
    count += 1
    with open(os.path.join(pdf_path, file), 'r', encoding='utf8') as f:
      section_list += ast.literal_eval(f.read()) # index : 0=기준명, 1=페이지, 2=소제목, 3=본문내용

print('Number of Files = ', count)
print('Number of Sections = ', len(section_list))

Number of Files =  2
Number of Sections =  1024


In [35]:
# Documents Embeddings 생성
docs_body_embeddings = embed_model.embed_documents([section[3] for section in section_list]) # 내용 embeddings
docs_subtitle_embeddings = embed_model.embed_documents([section[2] for section in section_list]) # 소제목 embeddings

# Documents Embeddings 확인
print('Vector size = ', len(docs_body_embeddings[0]))

Number of Sections =  1024
Vector size =  3072


In [38]:
# Query Embeddings 생성
query_embeddings = embed_model.embed_query(query)

# Query Embeddings 확인
print('Vector size = ', len(query_embeddings)) # vector size
print(query_embeddings[:5]) # vector (only first 5 elements)

Vector size =  3072
[0.03761359814459702, -0.01070817772512704, -0.002221496869908816, -0.005537807595760987, 0.013655176465189229]


In [41]:
# Cosine 유사도 계산 (소제목 & 본문내용)
cos_sim_body = np.dot(query_embeddings, np.array(docs_body_embeddings).T) / (np.linalg.norm(query_embeddings) * np.linalg.norm(docs_body_embeddings, axis=1))
cos_sim_subtitle = np.dot(query_embeddings, np.array(docs_subtitle_embeddings).T) / (np.linalg.norm(query_embeddings) * np.linalg.norm(docs_subtitle_embeddings, axis=1))

# 소제목 & 본문내용에 대한 Cosine 유사도 sum (out of 2)
cos_sim = cos_sim_body + cos_sim_subtitle

# sort (descending)
cos_sim_sorted_idx = np.argsort(cos_sim)[::-1]

# 결과 확인
for i in cos_sim_sorted_idx[:show_result]: # 보여줄 결과값 갯수 = 5개
  print(section_list[i][2], '\n')
  print(section_list[i][3])
  print('# References = ', section_list[i][0], ', Page.', section_list[i][1])
  print('# Cosine Similarity = ', cos_sim[i])
  print('-----------------------------')

###3###| 0701 | 일반사항 | 671 |기준 해설제7장  강구조
 

0701 일반사항 / 0702 설계요구사항 / 0703 골조의 안정성 / 0704 인장재 / 0705 압축재 / 0706 휨부재 /
0707 전단력을 받는 부재  / 0708 조합력과 비틀림을 받는 부재  / 0709 합성부재 / 0710 접합, 절점 및 파스너 / 
0711 강관구조접합 / 0712 사용성 설계 / 07 13 강구조의 내진설계 / 0714 합성구조의 내진설계 / 0715 제작, 설치 및 
품질관리 / 0716 비탄성해석 및 설계 / 0717 물고임에 대한 설계  / 0718 내화설계 / 0719 기존 구조물의 평가 / 
0720 기둥과 보의 안정용가새 / 0721 직접해석법 / 0722 내진성능검증, 품질확보계획 및 용접규정 / 0 723 냉간성형강구조
0701 일반사항
0701.1 적용범위
이 장은 구조용 강재를 사용한 건축물 및 공작물 (이하 강구조
물)에 적용한다 .
0701.1.1 내진설계기준의 비적용
반응수정계수 이 3 이하인 강구조물의 설계 , 제 작  및  설 치 는 
이 장을 따르고 , 0713과 0714의 내진설계기준을 적용하지 않는다 .
0701.1.2 내진설계기준의 적용
반응수정계수 이 3을 초과하는 강구조물의 설계는 0713과 
0714의 내진설계기준을 포함한 이 장의 요구사항을 만족하여야 
한다.
0701.2 용어의 정의
1점 집중하중： 부재의 플랜지에 직교방향으로 작용하는 인장력이
나 압축력
2점 집중하중： 부재의 한쪽 면에 1쌍으로 작용하는 동일한 힘
가새：골조에서 기둥과 기둥 간에 대각선상으로 설치한 사재로 
수평력에 대한 저항부재의 하나
가새골조： 횡력에 저항하기 위하여 건물골조시스템 또는 이중골
조시스템에서 사용하는 중심형 또는 편심형의 수직트러스 또는 
이와 동등한 구성체
가새실험체： 프로토타입의 가새를 모형화하기 위하여 실험에 사
용하는 단일의 좌굴방지가새0701 일반사항
0701.1 적용범위
한계상태설계법 (limit st